# Dependencies

In [636]:
#%pip install pyautogui opencv-python easyocr

In [637]:
#%pip install torch==2.3.1 torchaudio==2.3.1 torchvision==0.18.1

# Libraries

In [639]:
import os
import cv2
import time
import easyocr
import logging
import pyautogui
import numpy as np
from collections import deque
import matplotlib.pyplot as plt

In [640]:
start_time = time.time()

In [641]:
def take_screenshot():
    screenshot = pyautogui.screenshot()
    screenshot = np.array(screenshot)
    return cv2.cvtColor(screenshot, cv2.COLOR_RGB2BGR)

def get_colors():
    return np.array([
        [81, 215, 170],  # Green light
        [73, 209, 162],  # Green
        [159, 194, 229], # Nude light
        [153, 184, 215], # Nude
        [58, 175, 135]   # line
    ], dtype=np.uint8)

def create_mask_colors(screenshot, colors):
    return [cv2.inRange(screenshot, color, color) for color in colors]

def get_table_contours(masks):
    combined_mask = np.bitwise_or.reduce(masks)
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def get_table_dimensions(contours):
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    return x, y, w, h

In [642]:
screenshot = take_screenshot()
colors = get_colors()
masks = create_mask_colors(screenshot, colors)

table_contours = get_table_contours(masks)
table_dims = get_table_dimensions(table_contours)

square_dims = {
    (450, 360): 45, # Easy
    (540, 420): 30, # Medium
    (600, 500): 25, # Hard
}

square_size = square_dims[(table_dims[2], table_dims[3])]

In [643]:
logging.getLogger().setLevel(logging.ERROR)
reader = easyocr.Reader(['en'])

In [644]:
def get_square(screenshot, table_dims, i, j, square_size):
    dx, dy = j * square_size, i * square_size
    x, y, w, h = (
        table_dims[0] + dx, 
        table_dims[1] + dy, 
        square_size, 
        square_size
    )
    return screenshot[y:y+h, x:x+w]

def scale_image(square, width, inter=cv2.INTER_AREA):
    original_height, original_width = square.shape[:2]
    if original_width == width:
        return square
    
    ratio = width / float(original_width)
    height = int(original_height * ratio)
    
    return cv2.resize(square, (width, height), interpolation=inter)

def str_to_number(str_value):
    try:
        return int(str_value)
    except ValueError:
        return None

def get_number(square):
    gray_square = cv2.cvtColor(square, cv2.COLOR_BGR2GRAY)
    _, binarized_square = cv2.threshold(gray_square, 128, 255, cv2.THRESH_BINARY)
    results = reader.readtext(binarized_square)
    if not results:
        return None

    sorted_results = sorted(results, key=lambda x: x[2], reverse=True)
    return int(sorted_results[0][1])
    
def get_cell_value(square, colors):
    for index, color in enumerate(colors):
        mask = cv2.inRange(square, color, color)
        color_percentage = (np.sum(mask) / (mask.size * 255)) * 100
        
        if color_percentage > 90:
            return 0 if index < 2 else -1
    
    return get_number(square)

In [645]:
_, _, width, height = table_dims
table = np.zeros((height // square_size, width // square_size))

for i in range(len(table)):
    for j in range(len(table[i])):
        square = get_square(screenshot, table_dims, i, j, square_size)
        resized_square = scale_image(square, 4 * square_size)
        table[i, j] = get_cell_value(resized_square, colors[:4])
    print(f"Fila {i + 1} de {len(table)} leida")

Fila 1 de 14 leida
Fila 2 de 14 leida
Fila 3 de 14 leida
Fila 4 de 14 leida
Fila 5 de 14 leida
Fila 6 de 14 leida
Fila 7 de 14 leida
Fila 8 de 14 leida
Fila 9 de 14 leida
Fila 10 de 14 leida
Fila 11 de 14 leida
Fila 12 de 14 leida
Fila 13 de 14 leida
Fila 14 de 14 leida


In [646]:
def show_table(table):
    str_value = ""
    for row in table:
        for col in row:
            value = f" {float(col):.3f}"
            str_value += value + ""
        str_value += "\n"

    print(str_value)

In [647]:
show_table(table)

 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 3.000 1.000 2.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 2.000 -1.000 nan 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 0.000 0.000 0.000 0.000 0.000 2.000 1.000 -1.000 2.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 0.000 0.000 0.000 0.000 0.000 1.000 -1.000 -1.000 1.000 1.000 nan 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 0.000 0.000 0.000 0.000 0.000 2.000 1.000 1.000 -1.000 -1.000 2.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 2.000 -1.000 -1.000 1.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 nan -1.000 -1.000 1.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 3.000 -1.000 -1.000 1.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 

In [648]:
def get_cell(table, i, j):
    if 0 <= i < len(table) and 0 <= j < len(table[0]):
        return table[i][j]
    return -3

def get_adjacent_cells(table, i, j):
    adjacent_cells = []
    
    for dx, dy in [(-1, -1), (0, -1), (1, -1), (1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0)]:
        ni, nj = i + dy, j + dx
        cell_value = get_cell(table, ni, nj)
        
        if cell_value != -3:
            adjacent_cells.append((ni, nj, cell_value))
    
    return adjacent_cells

def adjacent_cell_with_positive_numbers(table, i, j):
    return [(ni, nj, cell_value) for ni, nj, cell_value in get_adjacent_cells(table, i, j) if cell_value > 0]

def num_adjacent_cell_with_positive_numbers(table, i, j):
    return sum(1 for ni, nj, cell_value in get_adjacent_cells(table, i, j) if cell_value > 0)

def mines_around_cell(table, i, j):
    return sum(1 for ni, nj, cell_value in get_adjacent_cells(table, i, j) if cell_value == -2)

def empty_cells_around(table, i, j):
    return {(ni, nj) for ni, nj, cell_value in get_adjacent_cells(table, i, j) if cell_value == 0}

def heuristic(table, i, j):
    mines_around, empty_cells = 0, set()
    adjacent_cells = adjacent_cell_with_positive_numbers(table, i, j)
    
    if not adjacent_cells or get_cell(table, i, j) != 0:
        return 0

    for ni, nj, cell_value in adjacent_cells:
        mines_around += cell_value - mines_around_cell(table, ni, nj)
        empty_cells.update(empty_cells_around(table, ni, nj))
        
    return mines_around / len(empty_cells) if empty_cells else int(mines_around > 0)

def is_special_case(table, i, j):
    cases = [
        [ # Arriba-Derecha
            [(0, -2), (1, -2), (2, -2), (2, -1), (2, 0)], # Bloqueadas
            [(0, -1), (1, 0)], # Diagonales con numeros
            (1, -1) # Diagonal con un 1
        ],
        [ # Derecha-Abajo
            [(2, 0), (2, 1), (2, 2), (1, 2), (0, 2)], # Bloqueadas
            [(1, 0), (0, 1)], # Diagonales con numeros
            (1, 1) # Diagonal con un 1
        ],
        [ # Abajo-Izquierda
            [(0, 2), (-1, 2), (-2, 2), (-2, 1), (-2, 0)], # Bloqueadas
            [(0, 1), (-1, 0)], # Diagonales con numeros
            (-1, 1) # Diagonal con un 1
        ],
        [ # Izquierda-Arriba
            [(-2, 0), (-2, -1), (-2, -2), (-1, -2), (0, -2)], # Bloqueadas
            [(-1, 0), (0, -1)], # Diagonales con numeros
            (-1, -1) # Diagonal con un 1
        ]
    ]

    if get_cell(table, i, j) == 0:
        for case in cases:
            num_lockeds = sum(1 for dx, dy in case[0] if -2 <= get_cell(table, i + dy, j + dx) <= -1)
            is_numbers = all(get_cell(table, i + dy, j + dx) > 0 for dx, dy in case[1])
            
            dx, dy = case[2]
            is_one = get_cell(table, i + dy, j + dx) == 1

            # Caso 1
            if num_lockeds >= 3 and is_numbers and is_one:
                return True

            # Caso 2
            is_ones = all(get_cell(table, i + dy, j + dx) == 1 for dx, dy in case[1])
            num_lockeds = sum(1 for dx, dy in case[0] if -2 <= get_cell(table, i + dy, j + dx) <= -1)
            if is_one and is_ones and num_lockeds > 2:
                return True

    return False

In [649]:
class TreeNode:
    def __init__(self, table):
        self.table = table
        self.children = []

    def find_paths(self):
        def dfs(node, leaves):
            if node is None:
                return

            if not node.children:
                leaves.append(node.table)

            for child in node.children:
                dfs(child, leaves)

        leaves = []
        dfs(self, leaves)
        return leaves

In [650]:
def calculate_heuristics(table):
    heuristics, special_case = [], None
    for i in range(len(table)):
        for j in range(len(table[0])):
            if is_valid_special_case(table, i, j):
                return None, (i, j)
                
            value = heuristic(table, i, j)
            if value > 0:
                heuristics.append((i, j, value))
    return heuristics, None

def is_valid_special_case(table, i, j):
    if is_special_case(table, i, j):
        new_table = [row.copy() for row in table]
        new_table[i][j] = -2
        return is_valid_board(new_table)
    return False

def is_valid_board(table, full=False):
    for i in range(len(table)):
        for j in range(len(table[i])):
            cell_value = get_cell(table, i, j)
            mines_around = mines_around_cell(table, i, j)
            adjacent_cells = adjacent_cell_with_positive_numbers(table, i, j)
            cond = (cell_value < mines_around) or (mines_around < cell_value and full)

            if adjacent_cells and cell_value > 0 and cond:
                return False
    return True

def can_continue(table):
    for i in range(len(table)):
        for j in range(len(table[i])):
            cell = get_cell(table, i, j)
            if 0 < cell and mines_around_cell(table, i, j) < cell:
                return True
    return False

def calculate_probability(matrices, target_number, row_index, col_index):
    total_matrices = len(matrices)
    count = sum(matrix[row_index][col_index] == target_number for matrix in matrices)
    return 0 if total_matrices == 0 else (count / total_matrices), count

def get_probabilities(table, paths, n, m):
    prob_table = np.ones((n, m))
    
    for i in range(n):
        for j in range(m):
            prob, count = calculate_probability(paths, -2, i, j)
            num_adj = num_adjacent_cell_with_positive_numbers(table, i, j)
            cell = get_cell(table, i, j)
            
            if 0 < count or (cell == 0 and 0 < num_adj):
                prob_table[i, j] = prob
            else:
                prob_table[i, j] = 1
                
    return prob_table

def find_min_position(matrix):
    np_matrix = np.array(matrix)
    return np.unravel_index(np.argmin(np_matrix), np_matrix.shape)

def find_best_movement(table, root):
    paths = root.find_paths()
    valid_matrices = [path for path in paths if is_valid_board(path, True)]
    print(len(paths), len(valid_matrices))
    prob_table = get_probabilities(table, valid_matrices, len(table), len(table[0]))
    show_table(prob_table)
    return find_min_position(prob_table) if valid_matrices else (0, 0)

def table_to_tuple(table):
    return tuple(tuple(row) for row in table)

def try_possibility(root, visited_tables, i, j):
    new_table = [row.copy() for row in root.table]
    new_table[i][j] = -2
    table_tuple = table_to_tuple(new_table)

    if is_valid_board(new_table) and table_tuple not in visited_tables:
        visited_tables.add(table_tuple)
        node = TreeNode(new_table)
        root.children.append(node)
        generate_possibilities(node, visited_tables)

def generate_possibilities(root, visited_tables=set()):
    heuristics_list, special_case = calculate_heuristics(root.table)
    if special_case:
        return try_possibility(root, visited_tables, special_case[0], special_case[1])
                
    ordered_heuristics_list = sorted(heuristics_list, key=lambda x: x[2], reverse=True)
    if not can_continue(root.table) or not ordered_heuristics_list:
        return

    for land_mine in ordered_heuristics_list:
        try_possibility(root, visited_tables, land_mine[0], land_mine[1])

In [ ]:
rootNode = TreeNode(table)
generate_possibilities(rootNode)

In [ ]:
square_i, square_j = find_best_movement(table, rootNode)

In [ ]:
def show_best_movement(table_dims, i, j, square_size):
    x, y, w, h = table_dims
    sx, sy = j * square_size, i * square_size
        
    table = screenshot[y:y+h, x:x+w]    
    cv2.rectangle(table, (sx, sy), (sx+square_size, sy+square_size), (0, 0, 0), 2)

    plt.imshow(cv2.cvtColor(table, cv2.COLOR_BGR2RGB), cmap='gray')
    plt.axis('off')  # Ocultar los ejes
    plt.show()

In [ ]:
show_best_movement(table_dims, square_i, square_j, square_size)

In [ ]:
end_time = time.time()

In [ ]:
elapsed_time = end_time - start_time

minutes = int(elapsed_time // 60)
seconds = int(elapsed_time % 60)

print(f"{minutes} min and {seconds} s")